In [ ]:
%matplotlib inline

# XOR 문제를 상수 W 값 MLP 예제
* 2개 층으로 구성된 NN으로 XOR 문제를 푸는 예시
* 단, W와 b를 이미 알고 있는 값을 지정해서 푼다.
* 이것이 가능하다면 여러 개층의 W와 b를 GD로 풀수 있으면 많은 문제를 딥러닝으로 해결이 가능하다는 말이다.
* 여기서 사용한 W와 b
![xor_nn_1](https://user-images.githubusercontent.com/661959/54298177-9e82f080-45fb-11e9-8bdd-1f86718c6f5d.png)
* $W$와 $bias$
    * Layer-1 y1: $W = \begin{bmatrix} 5 \\5 \end{bmatrix} , b= -8$
    * Layer-1 y2: $W = \begin{bmatrix}-7 \\ -7 \end{bmatrix} , b= 3$
    * Layer-2 : $W= \begin{bmatrix}-11 \\ -11 \end{bmatrix}, b = 6$
* 연산식
    * $x=(0,0)$
        * $ \begin{bmatrix}0 &0\ \end{bmatrix} \begin{bmatrix}5\\5\end{bmatrix}-8 = -8, sig(-8)=0$
        * $ \begin{bmatrix}0 &0\ \end{bmatrix} \begin{bmatrix}-7\\-7\end{bmatrix}+3 = 3, sig(3)=1$
            * $ \begin{bmatrix}0 &1\ \end{bmatrix} \begin{bmatrix}-11\\-11\end{bmatrix}+6 =-11+6= -5, sig(-5)=0$
    * $x=(0,1)$
        * $ \begin{bmatrix}0 &1\ \end{bmatrix} \begin{bmatrix}5\\5\end{bmatrix}-8 =5 -8=-3, sig(-3)=0$
        * $ \begin{bmatrix}0 &1\ \end{bmatrix} \begin{bmatrix}-7\\-7\end{bmatrix}+3 = -7+3, sig(-4)=0$
            * $ \begin{bmatrix}0 &1\ \end{bmatrix} \begin{bmatrix}-11\\-11\end{bmatrix}+6 =6, sig(6)=1$
    * $x=(1,0)$
        * $ \begin{bmatrix}1 &0\ \end{bmatrix} \begin{bmatrix}5\\5\end{bmatrix}-8 =5 -8=-3, sig(-3)=0$
        * $ \begin{bmatrix}1 &0\ \end{bmatrix} \begin{bmatrix}-7\\-7\end{bmatrix}+3 = -7+3, sig(-4)=0$
            * $ \begin{bmatrix}0 &1\ \end{bmatrix} \begin{bmatrix}-11\\-11\end{bmatrix}+6 =6, sig(6)=1$
    * $x=(1,1)$
        * $ \begin{bmatrix}0 &1\ \end{bmatrix} \begin{bmatrix}5\\5\end{bmatrix}-8 =5+5-8=2, sig(2)=1$
        * $ \begin{bmatrix}0 &1\ \end{bmatrix} \begin{bmatrix}-7\\-7\end{bmatrix}+3 = -7-7+3=-11, sig(-11)=0$
            * $ \begin{bmatrix}1 &0\ \end{bmatrix} \begin{bmatrix}-11\\-11\end{bmatrix}+6 =-11+6= -5, sig(-5)=0$
* 연산 결과

| $x_1$ $x_2$ | $y_1$ $y_2$ | $\hat y$|
|---|---|---|
|0,0|0,1|0
|0,1|0,0|1
|1,0|0,0|1
|1,1|1,0|0



## Multi-variable 형식으로 축소
![xor_nn_2](https://user-images.githubusercontent.com/661959/54298185-a347a480-45fb-11e9-91d4-e98111241794.png)

* $W$와 bias
    * $x = \begin {bmatrix}0 & 0\\ 0&1\\1&0\\1&1 \end {bmatrix}$
    * $W_1 = \begin{bmatrix}5&-7 \\ 5&-7\end{bmatrix}$
    * $b_1 = \begin{bmatrix}-8 & 3 \end{bmatrix}$
    * $W_2 = \begin{bmatrix}-11 \\ -11\end{bmatrix}$
    * $b_2 = 6 $
* 연산식
    * $ \hat y =
  \begin{cases}
    K(x) = sigmoid(Xw_1 + b_1)\\
    H(x) = sigmoid(K(x)W_2 + b_2)
  \end{cases}
    $

In [ ]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

W1 = tf.Variable(np.array([[5,-7], [5,-7]], dtype=np.float32), name='weight1')
b1 = tf.Variable(np.array([[-8, 3]], dtype=np.float32), name='bias1')
L1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(np.array([[-11],[-11]], dtype=np.float32), name='weight2')
b2 = tf.Variable(np.array([6], dtype=np.float32), name='bias2')
hypothesis = tf.sigmoid(tf.matmul(L1, W2) + b2)


# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
   # print(sess.run(hypothesis , feed_dict={X: x_data, Y: y_data}))
    # Accuracy report
    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
    )
    
    print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")


# Backpropagation
* Chain Rule
    * $f(g(x))$, $f(g)$, $g(x)$에 대해 미분
        * $\displaystyle \frac{\partial f}{\partial x} = \frac{\partial f}{\partial g}\frac{\partial g}{\partial x}$
![](https://user-images.githubusercontent.com/661959/54297147-9924a680-45f9-11e9-9cab-9930330a8f19.png)    
    

# XOR 문제 MLP 학습 예제
* 앞서 상수로 풀었던 XOR 문제를 학습해서 해결

In [ ]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
b1 = tf.Variable(tf.random_normal([2]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
b2 = tf.Variable(tf.random_normal([1]), name='bias2')
hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        _, cost_val = sess.run([train, cost], feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            print(step, cost_val)

    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
    )
    
    print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")


'''
Hypothesis:
[[0.01338216]
 [0.98166394]
 [0.98809403]
 [0.01135799]] 
Predicted:
[[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:
1.0
'''

# Vanishing Gradient
![](https://t1.daumcdn.net/cfile/tistory/997E1B4C5BB6EAF239)
* sigmoid 함수를 사용하면 input값들이(x1,x2,x3....xn) layer을 거치면서 0에 수렴
* 0에 수렴하는 값들이 다른 layer의 input 값으로 입력된다.
* 입력된 값들은 layer를 거치면서 0에 수렴
* x1,x2,x3....xn의 값은 최종으로 출력 되는 값에 영향이 없다
* cost가 줄어들지 않는다.

## ReLU Activation Function 
* Rectified Linear Unit
* `max(0,x)` 

# Weight 초기 값
* 0을 사용하지 말것
* RBM(Restricted Boltzmann Machine) 초창기 사용
* Xavier
    * `W = np.random.randn(in, out)/np.sqrt(in)`
    * 입력값과 출력 값 사이의 난수를 입력 값의 제곱근으로 나눈다.
* He
    * `W = np.random.randn(in, out)/np.sqrt(in/2)`
    * 입력 값을 2로 나눈 제곱근, 분모가 작아지기 때문에 xavier 보다 넓은 범위의 난수 
    
![](https://t1.daumcdn.net/cfile/tistory/2379CF4E57A0077534)